# Brief Projet 2 : Enquêter sur les mensonges de Donald Trump
## Sub compétence : scarpper un article de journal =>  vers 1df vers 1csv. 

url_lies : https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html  
Twitter Donald Trump 
url_t = https://twitter.com/realDonaldTrump    
- **Q : Réccuperer les tweets de Donald Trump (en vue d'une analyse par un data Scientist : popularité,...) ?**    
    
    
    - Les tweets est un flux, un Web Scraping s'impose ! Sinon, vs devez être toute la journée devant votre poste pr réccupérer ces fameux tweets (au fur et à mesure de leur apparition). Une autre piste est d'utiliser l'API de twitter (...).  
 

- **Q : Réccuperer, construire le df et l'exporter vers fichier csv :**
    - les dates des mensonges
    - les mensonges correspondants à chaque date
    - les argumentations de l'auteur qui correspondant à chaque mensonge
    - l'URL de l'argumentation

Le web scraping devra fournir ce df :

date | lie | explanation | url
--- | --- | --- | ---
Jan. 21, 2017 | I wasn't a fan of Iraq. I didn't want to go in... | He was for an invasion before he was against it. | https://www.buzzfeed.com/andrewkaczynski/in-20...
Jan. 21, 2017 | A reporter for Time magazine — and I have been... | Trump was on the cover 11 times and Nixon appe... | http://nation.time.com/2013/11/06/10-things-yo...
Jan. 23, 2017 | Between 3 million and 5 million illegal votes ... | There's no evidence of illegal voting. | https://www.nytimes.com/2017/01/23/us/politics...
Jan. 25, 2017 | Now, the audience was the biggest ever. But th... | Official aerial photos show Obama's 2009 inaug... | https://www.nytimes.com/2017/01/21/us/politics...

In [2]:
import requests
r = requests.get('https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html')
#print(r.text[0 : 500])

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all('span', attrs={'class':'short-desc'})
len(results)
results[0:3]
results[-1]

<span class="short-desc"><strong>Nov. 11 </strong>“I'd rather have him  – you know, work with him on the Ukraine than standing and arguing about whether or not  – because that whole thing was set up by the Democrats.” <span class="short-truth"><a href="https://www.nytimes.com/interactive/2017/12/10/us/politics/trump-and-russia.html" target="_blank">(There is no evidence that Democrats "set up" Russian interference in the election.)</a></span></span>

In [4]:
# Extraire la date
first_result = results[0]
first_result
first_result.find('strong')
first_result.find('strong').text
first_result.find('strong').text[0:-1] + ', 2017'

'Jan. 21, 2017'

In [5]:
# Extraire la mensonge
first_result
first_result.contents
first_result.contents[1]
first_result.contents[1][1:-2]

"I wasn't a fan of Iraq. I didn't want to go into Iraq."

In [6]:
# Extraire l'explication
first_result.contents[2]
first_result.find('a')
first_result.find('a').text[1:-1]

'He was for an invasion before he was against it.'

In [7]:
# Extracting the URL
first_result.find('a')
first_result.find('a')['href']

'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'

In [8]:
# Building the dataset 
records = []
for result in results:
    date = result.('strong').text[0:-1] + ', 2017'
    lie = result.contents[1][1:-2]
    explanation = result.find('a').text[1:-1]
    url = result.find('a')['href']
    records.append((date, lie, explanation, url))
len(records)    
records[0 : 3] 

[('Jan. 21, 2017',
  "I wasn't a fan of Iraq. I didn't want to go into Iraq.",
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'),
 ('Jan. 21, 2017',
  'A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.',
  'Trump was on the cover 11 times and Nixon appeared 55 times.',
  'http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/'),
 ('Jan. 23, 2017',
  'Between 3 million and 5 million illegal votes caused me to lose the popular vote.',
  "There's no evidence of illegal voting.",
  'https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html')]

In [10]:
import pandas as pd
df = pd.DataFrame(records, columns=['date', 'lie', 'explanation', 'url'])
df.head()

,date,lie,explanation,url
0,"Jan. 21, 2017",I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,"Jan. 21, 2017",A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,"Jan. 23, 2017",Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,"Jan. 25, 2017","Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,"Jan. 25, 2017",Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...
